# Exercises

<div class="alert alert-block alert-info">

**1. Euler-Cromer**

A simple change to the first-order Euler method is to update the position first and then use the new position to evalue the acceleration before updating the velocity.  In this sense, we are treating the velocity update implicitly.
    
</div>

<div class="alert alert-block alert-info">

**2. Chaos**

A double pendulum system is nonlinear and can exhibit chaotic behavior&mdash;the solution at some time in the future depends sensitively on the initial conditions.
    
</div>